In [15]:
import pandas as pd
from io import StringIO


In [12]:
df = pd.read_parquet('../data/raw/parquets/point_by_point_pbp.parquet')
df.head(20)


,match_id,set_id,game_id,point_id,home_point,away_point,point_description,home_point_type,away_point_type,home_score,away_score,serving,scoring
0,11998445,3,13,0,1,0,0,6,5,6,7,1,2
1,11998445,3,13,1,1,1,0,5,6,6,7,1,2
2,11998445,3,13,2,1,2,0,5,6,6,7,1,2
3,11998445,3,13,3,1,3,0,5,1,6,7,1,2
4,11998445,3,13,4,1,4,0,5,1,6,7,1,2
5,11998445,3,13,5,2,4,0,1,5,6,7,1,2
6,11998445,3,13,6,3,4,0,1,5,6,7,1,2
7,11998445,3,13,7,3,5,0,5,1,6,7,1,2
8,11998445,3,13,8,4,5,0,6,5,6,7,1,2
9,11998445,3,13,9,4,6,0,5,3,6,7,1,2


In [13]:
df.shape

(1254739, 13)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254739 entries, 0 to 1254738
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   match_id           1254739 non-null  int64 
 1   set_id             1254739 non-null  int64 
 2   game_id            1254739 non-null  int64 
 3   point_id           1254739 non-null  int64 
 4   home_point         1254739 non-null  object
 5   away_point         1254739 non-null  object
 6   point_description  1254739 non-null  int64 
 7   home_point_type    1254739 non-null  int64 
 8   away_point_type    1254739 non-null  int64 
 9   home_score         1254739 non-null  int64 
 10  away_score         1254739 non-null  int64 
 11  serving            1254739 non-null  int64 
 12  scoring            1254739 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 124.4+ MB


<h3>
2-What is the average height of the players? <h3>

In [25]:
import pandas as pd

# Load home players
df_home = pd.read_parquet('../data/raw/parquets/match_home_team.parquet',
                          columns=['player_id', 'height'])

# Load away players
df_away = pd.read_parquet('../data/raw/parquets/match_away_team.parquet',
                          columns=['player_id', 'height'])

# Combine datasets
df_all = pd.concat([df_home, df_away], ignore_index=True)

# Remove duplicate players
df_unique = df_all.drop_duplicates(subset='player_id')

# Calculate average height
average_height = df_unique['height'].dropna().mean()

print("✅ Average height of all unique players:", round(average_height, 2), "cm")
print("📌 Total unique players counted:", len(df_unique))


✅ Average height of all unique players: 1.82 cm
📌 Total unique players counted: 2495


<h3>
3-Which player has the highest number of wins?
<h3>

In [35]:
import pandas as pd

match_event = pd.read_parquet('../data/raw/parquets/match_event.parquet')
home_team = pd.read_parquet('../data/raw/parquets/match_home_team.parquet')
away_team = pd.read_parquet('../data/raw/parquets/match_away_team.parquet')


home_wins = match_event[match_event['winner_code'] == 1].merge(
    home_team[['match_id', 'player_id', 'full_name']],
    on='match_id',
    how='left'
)

away_wins = match_event[match_event['winner_code'] == 2].merge(
    away_team[['match_id', 'player_id', 'full_name']],
    on='match_id',
    how='left'
)


all_wins = pd.concat([home_wins[['player_id','full_name']], away_wins[['player_id','full_name']]])
win_counts = all_wins.groupby(['player_id','full_name']).size().reset_index(name='wins')


top_player = win_counts.sort_values('wins', ascending=False).head(1)
print(top_player)

      player_id              full_name  wins
1105   230049.0  Jianu, Filip Cristian    20


<h3>
6- Which country has produced the most successful tennis?
<h3>

In [36]:
import pandas as pd


match_event = pd.read_parquet('../data/raw/parquets/match_event.parquet')
home_team = pd.read_parquet('../data/raw/parquets/match_home_team.parquet')
away_team = pd.read_parquet('../data/raw/parquets/match_away_team.parquet')


home_wins = match_event[match_event['winner_code'] == 1].merge(
    home_team[['match_id', 'player_id', 'country']],
    on='match_id',
    how='left'
)


away_wins = match_event[match_event['winner_code'] == 2].merge(
    away_team[['match_id', 'player_id', 'country']],
    on='match_id',
    how='left'
)


all_wins = pd.concat([home_wins[['country']], away_wins[['country']]])


country_wins = all_wins.groupby('country').size().reset_index(name='wins')


top_country = country_wins.sort_values('wins', ascending=False).head(1)
print(top_country)

   country  wins
29  France   659


In [37]:
import pandas as pd


match_event = pd.read_parquet('../data/raw/parquets/match_event.parquet')
home_team = pd.read_parquet('../data/raw/parquets/match_home_team.parquet')
away_team = pd.read_parquet('../data/raw/parquets/match_away_team.parquet')


home_wins = match_event[match_event['winner_code'] == 1].merge(
    home_team[['match_id','country']],
    on='match_id',
    how='left'
)


away_wins = match_event[match_event['winner_code'] == 2].merge(
    away_team[['match_id','country']],
    on='match_id',
    how='left'
)


all_wins = pd.concat([home_wins[['country']], away_wins[['country']]])


france_wins = all_wins[all_wins['country'] == 'France'].shape[0]

print(f"Total wins by French players: {france_wins}")

Total wins by French players: 659


<h3>
How many sets are typically played in a tennis match?
<h3>

In [8]:
import pandas as pd


df = pd.read_parquet("../data/raw/parquets/statistics_statistics.parquet")


df = df.dropna(subset=["period"])


sets_per_match = df.groupby("match_id")["period"].nunique()


average_sets = sets_per_match.mean()

print(f"Average number of sets typically played in a tennis match: {average_sets:.2f}")

Average number of sets typically played in a tennis match: 3.28
